# Zeitmessung
Wenn wir über Performance sprechen, spielt die Zeitmessung eine zentrale Rolle. Deshalb ist es entscheidend, die richtige Messmethode auszuwählen. Dabei gibt es verschiedene Aspekte, die zu beachten sind.

Die zentralen Fragen lauten:
- Was messe ich?
- Wie messe ich?
  
Wir betrachten dazu ein Beispiel zur Vektor-Addition. 
Wir werfen einen Blick auf das Programm `zeitmessung.c`. Den dazugehörigen Quellcode können Sie ebenfalls einsehen. Dieses Programm führt eine Vektor-Addition aus. Als Erstes kompilieren wir das Programm:



In [1]:
!clang -o zeit.exe zeitmessung.c 

Jetzt führen wir das Programm mit dem Bash-Befehl `time` aus und messen damit, wie lange die Ausführung benötigt. Es gibt in der Regel drei Zeiten aus:

1. real: Die tatsächliche verstrichene Wanduhrzeit (Wall-Clock-Time), die vom Start bis zum Ende des Programms benötigt wird. Sie kann auch als Gesamtlaufzeit betrachtet werden.

2. user: Die gesamte CPU-Zeit, die das Programm in User-Mode verbraucht hat. Dies ist die Zeit, die der Prozess tatsächlich Rechenleistung beansprucht hat.

3. sys: Die CPU-Zeit, die das Programm im Kernel-Mode verbraucht hat. Dies ist die Zeit, in der der Kernel Systemaufrufe im Auftrag des Prozesses durchführt.

In [2]:
!bash -c "time ./zeit.exe"


real	0m0.087s
user	0m0.077s
sys	0m0.010s


Diese Zahlen geben uns bereits einen guten Einblick in die Geschwindigkeit des Programms. Die "real"-Zeit ist ein bedeutender Parameter zur Bewertung eines Programms, bezieht aber auch viele Dinge ein, die eine genaue Bewertung des Programmes schwierig machen, vor allem bei kleinen Beispielen.

Oftmals, insbesondere bei den kurzen Beispielen, die wir im Kurs behandeln, interessiert uns die Ausführungszeit spezifischer Code-Abschnitte. In solchen Fällen müssen wir den Code instrumentieren und Messpunkte hinzufügen. 

Wir wollen uns daher das Programm aus dem Beispiel im Detail ansehen und definieren.
Hier noch mal alle benötigten Funktionen: 

In [3]:
void vector_add_1(double *a, double *b, double *c, int n) {
  for (int i = 0; i < n; i++) {
    c[i] = a[i] + b[i];
  }
}


In [4]:
double *a = NULL;
double *b = NULL;
double *c = NULL;

Damit wir das Programm ausführen können, müssen wir zunächst den Speicher allozieren und die Daten initialisieren:

In [5]:
void init(int n) {

  a = (double *)calloc(n, sizeof(double));
  b = (double *)calloc(n, sizeof(double));
  c = (double *)calloc(n, sizeof(double));

  for (int i = 0; i < n; i++) {
    a[i] = rand() % 10;
    b[i] = rand() % 10;
    c[i] = 0;
  }
}

Mit folgender Funktion können wir am Ende alles wieder aufräumen:

In [6]:
void cleanup() {
  if (a) {
    free(a);
    a = NULL;
  }
  if (b) {
    free(b);
    b = NULL;
  }
  if (c) {
    free(c);
    c = NULL;
  }
}

Wir verwenden nun die `CLOCK_MONOTONIC`, um Zeitintervalle zu messen. Diese Uhr gibt eine kontinuierlich ansteigende Zeit seit einem undefinierten Startpunkt (oftmals dem Systemstart) zurück und ist nicht von externen Zeitänderungen beeinflusst. 

Da unser Hauptinteresse Zeitintervallen gilt, ist diese Art von Uhr für uns ideal. Die Zeit wird in der Struktur `timespec` zurückgegeben, die zwei Werte enthält: Sekunden und Nanosekunden. Um ein Zeitintervall in Mikrosekunden zu berechnen, können wir die folgende Funktion verwenden (der Header `time.h` wird benötigt):

In [7]:
#include <time.h>
double timespec_diff_to_milliseconds(struct timespec start,
                                     struct timespec end) {
  double milliseconds;

  milliseconds = double(end.tv_sec - start.tv_sec) *
                 1000.0; // Sekunden in Millisekunden umrechnen
  milliseconds += double(end.tv_nsec - start.tv_nsec) /
                  1000000.0; // Nanosekunden in Millisekunden umrechnen

  return milliseconds;
}

Nun messen wir einmal die Zeit unseres Programms hier:

In [8]:
struct timespec start, end;

clock_gettime(CLOCK_MONOTONIC, &start); // Starte Messung

int n = 2 << 20;
init(n);
vector_add_1(a, b, c, n);
cleanup();

clock_gettime(CLOCK_MONOTONIC, &end); // Beende Messung

double time = timespec_diff_to_milliseconds(start, end);
printf("total time is %f milliseconds\n", time);

total time is 84.647328 milliseconds


Wie wir sehen können, liegt die gemessene Zeit sehr nahe an der zuvor ermittelten real-time, wenn auch etwas geringer. Dies ist darauf zurückzuführen, dass der Overhead für das Laden des Programms entfällt.

Bei dieser Messung werden allerdings auch Aspekte wie das Allozieren, Initialisieren und Freigeben von Daten berücksichtigt, die für die Bewertung eines Algorithmus nicht relevant sind. Natürlich sollte dieser Overhead insgesamt nicht außer Acht gelassen werden. 

Wenn man jedoch primär an der Effizienz eines Algorithmus interessiert ist, sollte man sich auf die Messung dieser spezifischen Zeit konzentrieren. Wir messen daher nun nur die Zeit unserer `vector_add_1` Funktion:

In [9]:
struct timespec start, end;
int n = 2 << 20;
init(n);

clock_gettime(CLOCK_MONOTONIC, &start); // Starte Messung

vector_add_1(a, b, c, n);

clock_gettime(CLOCK_MONOTONIC, &end); // Beende Messung

cleanup();
double time = timespec_diff_to_milliseconds(start, end);
printf("total time is %f milliseconds\n", time);

total time is 6.195798 milliseconds


Wie wir feststellen können, ist die Zahl deutlich geringer als alles bisher Gemessene. Bei einem kleinen Beispiel wie unserem war allerdings zu erwarten, dass dies der Fall sein würde.

Wie Sie sicher gemerkt haben, braucht das Programm bei mehrfacher Ausführung unterschiedlich lange. Daher ist es immer sinnvoll, mehrere Messungen zu machen und den Mittelwert zu nehmen.  In anderen Fällen kann es auch sinnvoll sein, den minimalen oder den maximalen Wert zu nehmen. 

Man kann hier auch entweder das vollständige Programm wiederholen, oder nur die Funktion, die gemessen wurde. Beides hat Vor- und Nachteile. Alles zu wiederholen dauert länger, aber je nach Problemstellung können Cache-Effekte z.B. dazu führen, dass die Ergebnisse verfälscht werden.

Folgendes Beispiel initialisiert die Daten nur einmal vor allen Messungen:

In [10]:
struct timespec start, end;
double time = 0.0;
int repeat = 100;

int n = 2 << 20;
init(n);
for (int i = 0; i < repeat; i++) {
  clock_gettime(CLOCK_MONOTONIC, &start); // Starte Messung
  vector_add_1(a, b, c, n);
  clock_gettime(CLOCK_MONOTONIC, &end); // Beende Messung
  time += timespec_diff_to_milliseconds(start, end);
}
cleanup();

printf("total time is %f milliseconds\n", time / repeat);

total time is 6.429560 milliseconds


Folgendes Beispiel initialisiert die Daten vor jeder Messung:

In [11]:
struct timespec start, end;
double time = 0.0;
int repeat = 100;

int n = 2 << 20;
for (int i = 0; i < repeat; i++) {
  init(n);
  clock_gettime(CLOCK_MONOTONIC, &start); // Starte Messung
  vector_add_1(a, b, c, n);
  clock_gettime(CLOCK_MONOTONIC, &end); // Beende Messung
  time += timespec_diff_to_milliseconds(start, end);
  cleanup();
}

printf("total time is %f milliseconds\n", time / repeat);


total time is 6.564294 milliseconds


Am besten ist eine Mischung aus beiden: 

In [12]:
struct timespec start, end;
double time = 0.0;
int repeat = 10;

int n = 2 << 20;
for (int i = 0; i < repeat; i++) {

  init(n);
  for (int j = 0; j < repeat; j++) {
    clock_gettime(CLOCK_MONOTONIC, &start); // Starte Messung
    vector_add_1(a, b, c, n);
    clock_gettime(CLOCK_MONOTONIC, &end); // Beende Messung
    time += timespec_diff_to_milliseconds(start, end);
  }
  cleanup();
}

printf("total time is %f milliseconds\n", time / (repeat * repeat));

total time is 6.632488 milliseconds


Wir betrachten nun die Methoden, die Zeit zu messen. Wir kennen schon die Funktion `clock_gettime`. Diese unterstützt verschiedene Uhren: 
* CLOCK_REALTIME: Es misst die Wanduhrzeit seit der Ära 1970 (meist als "Epoch" bezeichnet, die um 00:00:00 UTC, 1. Januar 1970 beginnt). Die Zeit, die von dieser Uhr gemessen wird, kann aufgrund von NTP-Anpassungen oder anderen manuellen Eingriffen vorwärts oder rückwärts springen.
* CLOCK_MONOTONIC: Diese Uhr geht ständig vorwärts und wird nicht von NTP-Anpassungen oder manuellen Zeitänderungen beeinflusst. Sie ist nützlich für Zeitspannen, bei denen ein konstantes Fortschreiten der Zeit erforderlich ist.
* CLOCK_PROCESS_CPUTIME_ID: Misst die CPU-Zeit, die vom aktuellen Prozess verbraucht wird. Diese Uhr ist hilfreich, um die CPU-Zeit eines Prozesses und nicht die tatsächlich verstrichene Wanduhrzeit zu messen.
* CLOCK_THREAD_CPUTIME_ID: Misst die CPU-Zeit, die vom aktuellen Thread verbraucht wird.

Schauen wir uns einmal die Auflösung an:

In [13]:
struct timespec res, start, end;

clock_getres(CLOCK_REALTIME, &res);
printf("REALTIME: %ld, %ld\n", res.tv_sec, res.tv_nsec);
clock_getres(CLOCK_MONOTONIC, &res);
printf("MONOTONIC: %ld, %ld\n", res.tv_sec, res.tv_nsec);
clock_getres(CLOCK_PROCESS_CPUTIME_ID, &res);
printf("PROCESS_CPUTIME_ID: %ld, %ld\n", res.tv_sec, res.tv_nsec);
clock_getres(CLOCK_THREAD_CPUTIME_ID, &res);
printf("THREAD_CPUTIME_ID: %ld, %ld\n", res.tv_sec, res.tv_nsec);

REALTIME: 0, 1
MONOTONIC: 0, 1
PROCESS_CPUTIME_ID: 0, 1
THREAD_CPUTIME_ID: 0, 1


Als Nächstes wollen wir uns den Overhead der Funktionen ansehen. Wir betrachten zudem noch die `clock()` Funktion. Die Rückgabe von `clock()` ist vom Typ `clock_t`, und der Wert repräsentiert die Anzahl der "Ticks", die seit dem Programmstart vergangen sind. Der Wert muss durch die Konstante CLOCKS_PER_SEC geteilt werden, um die verstrichene Zeit in Sekunden zu berechnen. 

Beachten Sie, dass diese Funktion nicht die Wanduhrzeit (d.h. die tatsächliche verstrichene Zeit) misst, sondern nur die CPU-Zeit, die vom Programm verbraucht wurde.

In [14]:
clock_gettime(CLOCK_REALTIME, &start);
clock_gettime(CLOCK_REALTIME, &end);
printf("REALTIME took %ld ns\n", end.tv_nsec - start.tv_nsec);

clock_gettime(CLOCK_MONOTONIC, &start);
clock_gettime(CLOCK_MONOTONIC, &end);
printf("MONOTONIC took %ld ns\n", end.tv_nsec - start.tv_nsec);

clock_gettime(CLOCK_PROCESS_CPUTIME_ID, &start);
clock_gettime(CLOCK_PROCESS_CPUTIME_ID, &end);
printf("PROCESS CPU took %ld ns\n", end.tv_nsec - start.tv_nsec);

clock_gettime(CLOCK_THREAD_CPUTIME_ID, &start);
clock_gettime(CLOCK_THREAD_CPUTIME_ID, &end);
printf("THREAD CPU took %ld ns\n", end.tv_nsec - start.tv_nsec);

clock_t start_t, end_t;
double total_n;
start_t = clock();
end_t = clock();
total_n = (double) (end_t - start_t) / CLOCKS_PER_SEC * 1e9;
printf("Clock took %lf ns\n", total_n);

REALTIME took 38 ns
MONOTONIC took 33 ns
PROCESS CPU took 2548 ns
THREAD CPU took 181 ns
Clock took 0.000000 ns


Wie wir sehen, hat die `clock()` Funktion den geringsten Overhead, zumindest wenn es um die CPU-Zeit geht. Messen wir noch mal unsere Vektor-Funktion: 

In [15]:
n = 2 << 20;
cleanup();
init(n);
vector_add_1(a, b, c, n);

repeat = 10;

clock_gettime(CLOCK_REALTIME, &start);
for (int i = 0; i < repeat; i++) {
  vector_add_1(a, b, c, n);
}
clock_gettime(CLOCK_REALTIME, &end);

printf("REALTIME took %f ms\n",
       timespec_diff_to_milliseconds(start, end) / repeat);
clock_gettime(CLOCK_MONOTONIC, &start);
for (int i = 0; i < repeat; i++) {
  vector_add_1(a, b, c, n);
}
clock_gettime(CLOCK_MONOTONIC, &end);
printf("MONOTONIC took %f ms\n",
       timespec_diff_to_milliseconds(start, end) / repeat);

clock_gettime(CLOCK_PROCESS_CPUTIME_ID, &start);
for (int i = 0; i < repeat; i++) {
  vector_add_1(a, b, c, n);
}
clock_gettime(CLOCK_PROCESS_CPUTIME_ID, &end);

printf("CPU_PROCESS took %f ms\n",
       timespec_diff_to_milliseconds(start, end) / repeat);

clock_gettime(CLOCK_PROCESS_CPUTIME_ID, &start);
for (int i = 0; i < repeat; i++) {
  vector_add_1(a, b, c, n);
}
clock_gettime(CLOCK_PROCESS_CPUTIME_ID, &end);
printf("THREAD_PROCESS took %f ms\n",
       timespec_diff_to_milliseconds(start, end) / repeat);

clock_t start_t, end_t;

start_t = clock();
for (int i = 0; i < repeat; i++) {
  vector_add_1(a, b, c, n);
}
end_t = clock();

total_n = (double) (end_t - start_t) / CLOCKS_PER_SEC * 1e3;
printf("Clock took %lf ms\n", total_n / repeat);

cleanup();


REALTIME took 6.459849 ms
MONOTONIC took 6.379838 ms
CPU_PROCESS took 6.488839 ms
THREAD_PROCESS took 6.510846 ms
Clock took 6.480900 ms


Die gemessenen Zeiten liegen alle eng beieinander, nur die CPU-Time ist in der Regel etwas geringer.  Nun möchten wir diese Messungen für ein paralleles Programm wiederholen, indem wir die Vektoraddition parallelisieren. Die Details dazu finden Sie in der 3. Kurseinheit. Wichtig für Sie ist, dass die Addition auf vier Threads verteilt wird.

In [16]:
#include <omp.h>
#pragma cling load("libomp.so")

In [17]:
void vector_add_2(double *a, double *b, double *c, int n) {
#pragma omp parallel for num_threads(4)
  for (int i = 0; i < n; i++) {
    c[i] = a[i] + b[i];
  }
}

In [18]:
#pragma omp parallel num_threads(4)
printf("Hello thread %d of %d \n", omp_get_thread_num(), omp_get_num_threads());

Hello thread 2 of 4 
Hello thread 0 of 4 
Hello thread 1 of 4 
Hello thread 3 of 4 


In [19]:
n = 2 << 20;
cleanup();
init(n);
vector_add_2(a, b, c, n);

repeat = 10;

clock_gettime(CLOCK_REALTIME, &start);
for (int i = 0; i < repeat; i++) {
  vector_add_2(a, b, c, n);
}
clock_gettime(CLOCK_REALTIME, &end);
printf("REALTIME took %f ms\n",
       timespec_diff_to_milliseconds(start, end) / repeat);

clock_gettime(CLOCK_MONOTONIC, &start);
for (int i = 0; i < repeat; i++) {
  vector_add_2(a, b, c, n);
}
clock_gettime(CLOCK_MONOTONIC, &end);
printf("MONOTONIC took %f ms\n",
       timespec_diff_to_milliseconds(start, end) / repeat);

clock_gettime(CLOCK_PROCESS_CPUTIME_ID, &start);
for (int i = 0; i < repeat; i++) {
  vector_add_2(a, b, c, n);
}
clock_gettime(CLOCK_PROCESS_CPUTIME_ID, &end);
printf("CPU_PROCESS took %f ms\n",
       timespec_diff_to_milliseconds(start, end) / repeat);

clock_gettime(CLOCK_PROCESS_CPUTIME_ID, &start);
for (int i = 0; i < repeat; i++) {
  vector_add_2(a, b, c, n);
}
clock_gettime(CLOCK_PROCESS_CPUTIME_ID, &end);
printf("THREAD_PROCESS took %f ms\n",
       timespec_diff_to_milliseconds(start, end) / repeat);

clock_t start_t, end_t;
double total_n;
start_t = clock();
for (int i = 0; i < repeat; i++) {
  vector_add_2(a, b, c, n);
}
end_t = clock();
total_n = (double) (end_t - start_t) / CLOCKS_PER_SEC * 1e3;
printf("Clock took %lf ms\n", total_n / repeat);

REALTIME took 1.904912 ms
MONOTONIC took 1.853533 ms
CPU_PROCESS took 7.003647 ms
THREAD_PROCESS took 6.948687 ms
Clock took 7.100700 ms


Wie wir sehen können, messen wir unterschiedliche Zeiten. Aber welche Zeit ist nun die "richtige"?
Das hängt von der Perspektive ab. 

Wenn wir an der Wanduhrzeit ("Wall-Time") interessiert sind, also wie lange die Berechnung tatsächlich in Echtzeit gedauert hat, dann ist die MONOTONIC-Clock die zuverlässigste Wahl.

Die Zeit, die mit clock() gemessen wurde, hat eine andere Bedeutung. Sie gibt an, wie viele CPU-Zyklen für die Berechnung verwendet wurden. Im Idealfall nutzen wir jedoch vier CPU-Kerne für die Berechnung. Aufgrund des Overheads wird dieser Wert also vermutlich höher ausfallen.
PROCESS_CPUTIME und THREAD_CPUTIME sollten Sie bei parallelen Programmen nicht verwenden.

Am sinnvollsten ist es, die durch das Framework bereitgestellten Timer zu nutzen.